In [1]:
!pip install -q openai gradio

In [2]:
import openai

openai.api_key = "" ## 삭제 요망

# 답변생성 함수

In [5]:
def gen(x):
  gpt_prompt = [{
      "role": "system",
      "content":"당신은 친절하고 잘알려주는 인공지능 챗봇입니다. 입력에 대해 핵심과 친절하게 잘 대답해주세요."
  }]

  gpt_prompt.append({
      "role": "user",
      "content": x
  })

  gpt_response = openai.ChatCompletion.create(
      model = "gpt-3.5-turbo",
      messages = gpt_prompt
  )

  return gpt_response["choices"][0]["message"]["content"]

In [6]:
gen("안녕하세요, 너는 누구야?")

'안녕하세요! 저는 여러분의 질문에 친절하고 정확하게 답변해 주는 인공지능 챗봇입니다. 궁금한 것이 있으시면 언제든지 물어보세요! 😊'

In [7]:
gen("ChatGPT가 잘하는 것은 무엇이야?")

'ChatGPT는 자연어 처리 및 이해 기술을 이용하여 사용자의 질문에 답변을 생성하고 대화를 이어나가는 데 뛰어난 성능을 보입니다. 또한 다양한 주제에 대해 대화할 수 있으며 사용자와의 대화를 통해 학습하여 점차적으로 더 유창하고 의미 있는 대화를 할 수 있게 됩니다.그래서 ChatGPT는 일상적인 대화부터 정보 제공, 질문에 대한 답변, 조언 제공 등 다양한 상황에 유용하게 활용될 수 있습니다.'

# Gradio

In [8]:
import gradio as gr

def inference(text):
  return gen(text)

demo = gr.Interface(fn=inference, inputs="text", outputs="text")

demo.launch(debug=True, share=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://a93fca3b5c2ed7c255.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://a93fca3b5c2ed7c255.gradio.live


# 문맥 맥락 확장 및 기억

In [9]:
gpt_prompt = [{
    "role": "system",
    "content": "당신은 친절한 인공지능 챗봇입니다. 입력값에 대해 핵심과 간결하게 대답해주세요.",
}, {
    "role": "user",
    "content": "겨울에 먹기 좋은 과일을 추천해줘"
}, {
    "role": "assistant",
    "content": "겨울에는 망고, 포도, 거봉, 복숭아 등이 따뜻하고 맛있게 먹을 수 있는 과일입니다.",
}, {
    "role": "user",
    "content": "위 문장으로 영어로 번역해줘",
}]

gpt_response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=gpt_prompt
)

gpt_response["choices"][0]["message"]["content"]

'In winter, fruits such as mango, grapes, persimmons, and peaches are recommended for their warmth and deliciousness.'

# 챗봇 Gradio

In [14]:
import gradio as gr

def predict(input, history):
  history.append({"role": "user", "content": input})

  gpt_response = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=history
  )

  response = gpt_response["choices"][0]["message"]['content']
  history.append({"role":"assistant", "content":response})
  messages = [(history[i]["content"], history[i+1]["content"]) for i in range(1, len(history), 2)]

  return messages, history

with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label="ChatBot")

    state = gr.State([{
        "role":"system",
        "content":"당신은 아주 친절한 인공지능 챗봇입니다. 입력값에 대해 핵심으로 친절하게 대답해주세요."
    }])

    with gr.Row():
        # [수정된 부분] .style()을 제거하고 container=False를 인자로 넣었습니다.
        # scale=7을 추가하여 입력창이 버튼보다 넓게 보이도록 조정했습니다.
        txt = gr.Textbox(
            show_label=False,
            placeholder="챗봇에게 아무거나 물어보세요",
            container=False,
            scale=7
        )

    # 입력 후 엔터를 쳤을 때 실행
    txt.submit(predict, [txt, state], [chatbot, state])

    # (선택사항) 전송 후 입력창을 비우고 싶다면 아래 코드를 추가하세요
    # txt.submit(lambda: "", None, txt)

demo.launch(debug=True, share=True)

/tmp/ipython-input-2769502868.py:18: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="ChatBot")
/tmp/ipython-input-2769502868.py:18: DeprecationWarning: The default value of 'allow_tags' in gr.Chatbot will be changed from False to True in Gradio 6.0. You will need to explicitly set allow_tags=False if you want to disable tags in your chatbot.
  chatbot = gr.Chatbot(label="ChatBot")


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://4a7fb5003c45a3473e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://4a7fb5003c45a3473e.gradio.live


In [ ]:
안ㄴ